In [1]:
import os,sys
sys.path.append('../..')
sys.path.append('../../../gempy/')
import sandbox.sandbox as sb

import gempy as gp
#import gempy_sandbox a   s gpsb
import numpy as np
#from PIL import Image, ImageDraw
import pickle

../../../gempy/gempy/plotting/visualization.py:36: UserWarning: Steno 3D package is not installed. No 3D online visualization available.
  warnings.warn('Steno 3D package is not installed. No 3D online visualization available.')
../../../gempy/gempy/posterior_analysis.py:28: UserWarning: pymc (v3) package is not installed. No support for stochastic simulation posterior analysis.
  warnings.warn("pymc (v3) package is not installed. No support for stochastic simulation posterior analysis.")
../../sandbox/sandbox.py:34: RuntimeWarning: Two kernels cannot access the kinect at the same time. This will lead to a sudden death of the kernel. Be sure no other kernel is running before initialize a kinect object.
  'Be sure no other kernel is running before initialize a kinect object.', RuntimeWarning)


### Set up the sandbox 

First we activate the kinect

In [2]:
kinect = sb.Kinect(dummy=True)

dummy mode. get_frame() will return a synthetic depth frame, other functions may not work


Now we can set up the projector settings:

In [3]:
projector = sb.Projector(resolution = (1024,768), work_directory='./temp',
                        refresh=100, input_rescale=False)

Next is to pass the calibration---kinect-projector---to the projector class to be able to project just on the sandbox

In [4]:
calibration_filename = "sandbox_brisbane.dat" 
calibration = sb.Calibration(projector)
calibration.load(calibration_file=calibration_filename)
projector.set_calibration(calibration)
projector.start_stream()   

calibration data file not found. Using default values


Also we need to set up the general options of Matplotlib (the library to create the models). Here we are passing the colors of GemPy simply for consistency.

In [5]:
plot_im = sb.Plot(
    calibration=calibration,
    cmap=gp.plotting.colors.cmap,
    norm=gp.plotting.colors.norm
    )

### Setup the model gempy model

Now we need to generate the map that we want to poject. In the case of a geological map we will use GemPy.

First we need to load the underlaying 3D model. We can find some examples in the GemPy library

In [6]:
geo_model = gp.read_pickle(r'../../../gempy/notebooks/examples/Models_Lisa/lisa-1.pickle')

We want to scale the sandbox to the model extent. We can see a gempy model extent by

In [7]:
geo_model._geo_data.extent

array([   0, 2000,    0, 2000,    0, 1600])

In [8]:
# pass that extent
model_extent = geo_model._geo_data.extent

### Create sandbox grid following the right scale sandbox-model

Now we need to connect the cloud of points recorded by the kinect to the scale of geological model

In [9]:
scale=sb.Scale(extent=model_extent, calibration=calibration) #initialize scale object to handle the scaling of the model in the andbox
scale.calculate_scales()
grid = sb.Grid(calibration=calibration,scale=scale)

Aspect ratio of the model is fixed in XY
Model size is limited by Y dimension
scale in Model units/ mm (X,Y,Z): [2.666666666666667, 3.3333333333333335, 2.6666666666666665]
the shown extent is [0.0, 2662.5, 0.0, 1995.8333333333335] 


### Create the GeoMap module for the Sandbox

To increase the flexibility of the sandbox and generalize the hyperthreading (i.e. getting the sandbox running without blocking python), we have create a modules systems. This modules are classes that encapsulate the specific functionality for each type of map that we want to render. Lets create the geological map module.

In [10]:
geo_module = sb.GeoMapModule(geo_model, grid, plot_im)

In [11]:
geo_module.compute_model(kinect.get_filtered_frame())

/home/miguel/anaconda3/lib/python3.6/site-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


(array([[1., 1., 1., ..., 4., 4., 4.],
        [1., 1., 1., ..., 4., 4., 4.],
        [1., 1., 1., ..., 4., 4., 4.],
        ...,
        [7., 1., 1., ..., 4., 4., 4.],
        [7., 1., 1., ..., 4., 4., 4.],
        [7., 1., 1., ..., 4., 4., 4.]], dtype=float32),
 array([], shape=(0, 307200), dtype=float32))

### Hyperthreading

In [12]:
s_thread = sb.SandboxThread(geo_module, kinect, projector)

In [13]:
s_thread.run()

### set up the visualization

In [25]:
x = range(scale.output_res[0])
y = range(scale.output_res[1])

geol_map = sb.Plot(
    calibration=calibration,
    cmap=gp.plotting.colors.cmap,
    norm=gp.plotting.colors.norm
    )

fault_line = sb.Contour(
    start=0.5,
    end=2.5,
    step=1.0,
    linewidth=3.0,
    colors=[(1.0, 1.0, 1.0, 1.0)]
    )

main_contours = sb.Contour(
    start=scale.extent[4],
    end=scale.extent[5],
    step=100,
    show_labels=True
    )

sub_contours = sb.Contour(     

    start=scale.extent[4],
    end=scale.extent[5],
    step=25,
    linewidth=0.8,
    colors=[(0, 0, 0, 0.7)],
   )



 

In [42]:
class SandboxThread(threading.Thread):
    def __init__(self, t, *args):
        #self.sandbox_thread = threading.Thread.__init__(self, target=t, args=args)
        self.sandbox_lock = threading.Lock()
        self.
    

In [37]:
import threading
sandbox_lock = threading.Lock()
sandbox_thread= threading.Thread(target=run, daemon=None)

In [13]:
kinect.get_filtered_frame(n_frames=3,sigma_gauss=4)

array([[800., 803., 807., ..., 788., 792., 796.],
       [800., 803., 807., ..., 788., 792., 796.],
       [800., 803., 807., ..., 788., 792., 796.],
       ...,
       [800., 803., 807., ..., 788., 792., 796.],
       [800., 803., 807., ..., 788., 792., 796.],
       [800., 803., 807., ..., 788., 792., 796.]])

In [38]:
#new structure
def run():
    grid.update_grid(kinect.get_filtered_frame(n_frames=3,sigma_gauss=4))
    while True:
        with sandbox_lock:
            render_geological_map(geo_model, grid, geol_map)
        projector.show(rescale=False)
        

In [41]:
def render_geological_map(geo_model, grid, geol_map):
    sol = gp.compute_model_at(grid.depth_grid, geo_model)
    lith_block = sol[0][0]
    fault_blocks = sol[1][0::2]
    block = lith_block.reshape((scale.output_res[1], scale.output_res[0]))
    geol_map.render_frame(block)

    elevation = grid.depth_grid.reshape((scale.output_res[1], scale.output_res[0], 3))[:, :, 2]
    #fault_data = sol.fault_blocks.reshape((scalgeol_map.outfilee.output_res[1], scale.output_res[0])) 
    for fault in fault_blocks:
        fault=fault.reshape((scale.output_res[1],scale.output_res[0]))
        geol_map.add_contours(fault_line, [x,y,fault])


    geol_map.add_contours(main_contours,[x,y,elevation])
    geol_map.add_contours(sub_contours,[x,y,elevation])
    geol_map.save(outfile="./temp/current_frame.png")
    

In [39]:
import threading
sandbox_thread= threading.Thread(target=run, daemon=None)

In [40]:
sandbox_thread.run()

/home/miguel/anaconda3/lib/python3.6/site-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


In [33]:
while True:
    run()

KeyboardInterrupt: 

In [10]:
projector.start_stream()

In [ ]:
geology.lock.acquire()
print('na')
geology.lock.release()

In [ ]:
fault_line.show=True

x

In [ ]:
kinect.sigma_gauss=7

### Toggle fault visibility

In [ ]:
geology.pause()
if geology.module.show_faults is True:
    geology.module.show_faults = False
else:
    geology.module.show_faults = True
geology.resume()
    

## change contour levels 

In [ ]:
       self.fig = plt.figure(figsize=(self.w, self.h), dpi=100, frameon=False)
        self.ax = plt.Axes(self.fig, [0., 0., 1., 1.])
        self.ax.set_axis_off()
        self.fig.add_axes(self.ax)

geology.pause()
start = 0
end = 2000
main_step = 50
sub_step = 10

geology.module.main_contours = np.arange(start, end, main_step)
geology.module.sub_contours = np.arange(start, end, sub_step)
geology.resume()

In [ ]:
geology.lock.acquire()
geology.lock.release()


In [ ]:
geo_model_3d = gp.load_model('./gempy/notebooks/tutorials/saves/ch1-2.pickle')


In [ ]:
for i in range(10):
    with geology.lock:
        gp.plot.plot_section(geo_model_3d, 10)

In [ ]:
with geology.lock:
    for i in range(10):
        gp.plot.plot_section(geo_model_3d, 10)

In [ ]:
geology.lock.release()

In [ ]:
geology.lock.locked()

In [ ]:
geology.run()

In [ ]:
a = 0
for i in range(100000):
    a += i


In [ ]:
with geology.lock:
    for i in range(10):
        gp.compute_model(geo_model_3d)

In [ ]:
geo_model.interpolator.theano_graph

In [ ]:
for i in range(10):
    gp.compute_model(geo_model_3d)

In [ ]:
geo_model_3d.solutions

In [ ]:
gp.plot.plot_surfaces_3D(geo_model_3d)
